In [1]:
import numpy as np
import pandas as pd
import rasterio
from pyproj import Transformer

def raster_to_xyz_df(raster_path):
    with rasterio.open(raster_path) as src:
        band = src.read(1)
        nodata = src.nodata
        if nodata is not None:
            band = band.astype(float)
            band[band == nodata] = np.nan

        rows, cols = np.indices(band.shape)
        xs, ys = rasterio.transform.xy(src.transform, rows, cols)
        
        xs = np.array(xs).flatten()
        ys = np.array(ys).flatten()
        zs = band.flatten()

        df = pd.DataFrame({'x': xs, 'y': ys, 'z': zs})
        return df

def read_xyz(raster_path, default_band_name='Band'):
    with rasterio.open(raster_path) as src:
        bands = src.read()  # shape: (bands, height, width)
        x = np.linspace(src.bounds.left, src.bounds.right, src.width)
        y = np.linspace(src.bounds.top, src.bounds.bottom, src.height)
        crs = src.crs
        nodata = src.nodata
        band_names = [
            desc if desc else f'{default_band_name}_{i+1}' 
            for i, desc in enumerate(src.descriptions or [None]*src.count)
        ]
    return x, y, bands, crs, band_names, nodata

def xyz_to_dataframe(x, y, bands, band_names, nodata):
    xx, yy = np.meshgrid(x, y)
    flat_x = xx.ravel()
    flat_y = yy.ravel()

    data = {'x': flat_x, 'y': flat_y}
    for i, band in enumerate(bands):
        band_data = band.ravel().astype(np.float32)
        if nodata is not None:
            band_data[band_data == nodata] = np.nan
        data[band_names[i]] = band_data

    return pd.DataFrame(data)

def reproject_to_latlon(df, crs_from, crs_to='EPSG:4326'):
    transformer = Transformer.from_crs(crs_from, crs_to, always_xy=True)
    lon, lat = transformer.transform(df['x'].values, df['y'].values)
    df['lon'] = lon
    df['lat'] = lat
    cols = ['lat', 'lon'] + [col for col in df.columns if col not in ['x', 'y', 'lat', 'lon']]
    return df[cols]

def raster_to_dataframe(raster_path, default_band_name='Band', crs_to='EPSG:4326'):
    x, y, bands, crs, band_names, nodata = read_xyz(raster_path, default_band_name)
    df = xyz_to_dataframe(x, y, bands, band_names, nodata)
    df = reproject_to_latlon(df, crs, crs_to)
    return df